## Import

In [81]:
import cv2

## Initialization

In [88]:
dataset = [
    ("./dataset/00/l.png", "./dataset/00/r.png"),
    ("./dataset/01/l.png", "./dataset/01/r.png"),
    ("./dataset/02/l.png", "./dataset/02/r.png"),
    ("./dataset/03/l.png", "./dataset/03/r.png"),
    ("./dataset/04/l.png", "./dataset/04/r.png"),
    ("./dataset/05/l.png", "./dataset/05/r.png")
]

## Preprocessing

In [89]:
def apply_preprocessing(image):
    # Apply Gaussian Blur
    blurred_image = cv2.GaussianBlur(image, (5, 5), 0)
    # Apply Histogram Equalization
    equalized_image = cv2.equalizeHist(blurred_image)
    return equalized_image

## Block Matching

In [97]:
def compute_depth_map_bm(left_image, right_image):
    # numDisparities must be a multiple of 16.
    numDisparities = 128
    blockSize = 21
    
    stereo = cv2.StereoBM_create(numDisparities=numDisparities, blockSize=blockSize)
    disparity = stereo.compute(left_image, right_image)
    
    # Normalize depth map and convert to uint8
    min_disparity = disparity.min()
    max_disparity = disparity.max()
    
    if max_disparity - min_disparity > 0:
        depth_map = cv2.normalize(disparity, disparity, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
    else:
        depth_map = disparity
    
    depth_map = np.uint8(depth_map)
    
    # Apply color map
    depth_map_colored = cv2.applyColorMap(depth_map, cv2.COLORMAP_JET)
    
    return depth_map_colored

## Semi Global Matching

In [100]:
def compute_depth_map_sgm(left_image, right_image):
    # numDisparities must be a multiple of 16.
    numDisparities = 128
    blockSize = 5
    stereo = cv2.StereoSGBM_create(minDisparity=0,
                                   numDisparities=numDisparities,
                                   blockSize=blockSize,
                                   P1=8 * 3 * blockSize ** 2,
                                   P2=32 * 3 * blockSize ** 2,
                                   disp12MaxDiff=1,
                                   uniquenessRatio=10,
                                   speckleWindowSize=100,
                                   speckleRange=32,
                                   preFilterCap=63,
                                   mode=cv2.STEREO_SGBM_MODE_SGBM_3WAY)
    disparity = stereo.compute(left_image, right_image).astype(np.float32) / 16.0

    # Normalize depth map and convert to uint8
    min_disparity = disparity.min()
    max_disparity = disparity.max()
    
    if max_disparity - min_disparity > 0:
        depth_map = cv2.normalize(disparity, disparity, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
    else:
        depth_map = disparity
    
    depth_map = np.uint8(depth_map)
    
    # Apply color map
    depth_map_colored = cv2.applyColorMap(depth_map, cv2.COLORMAP_JET)
    
    return depth_map_colored

## Main

In [98]:
# Block Matching
for idx, (left_image_path, right_image_path) in enumerate(dataset):
    left_image = cv2.imread(left_image_path, cv2.IMREAD_GRAYSCALE)
    right_image = cv2.imread(right_image_path, cv2.IMREAD_GRAYSCALE)
    
    left_image = apply_preprocessing(left_image)
    right_image = apply_preprocessing(right_image)
    
    depth_map_bm = compute_depth_map_bm(left_image, right_image)
    cv2.imwrite(f'./result/block_matching/{idx}.png', depth_map_bm)

In [101]:
# Semi Global Matching
for idx, (left_image_path, right_image_path) in enumerate(dataset):
    left_image = cv2.imread(left_image_path, cv2.IMREAD_GRAYSCALE)
    right_image = cv2.imread(right_image_path, cv2.IMREAD_GRAYSCALE)
    
    left_image = apply_preprocessing(left_image)
    right_image = apply_preprocessing(right_image)
    
    depth_map_sgm = compute_depth_map_sgm(left_image, right_image)
    cv2.imwrite(f'./result/semi_global_matching/{idx}.png', depth_map_sgm)